In [9]:
import pandas as pd
import numpy as np
import vcf
import time
import re

In [10]:
class Pileup_line:
    def __init__(self, line): 
        split_line = line.split("\t")
        self.sequence = split_line[0]
        self.position = split_line[1]
        self.ref_base = split_line[2]
        self.read_count = split_line[3]
        self.read_results = split_line[4]
        self.base_quality = split_line[5]
        self.mapping_quality = split_line[6]
        
    # Pomocna funkcija za racunanje kvaliteta baze iz odgovarajuceg ascii karaktera
    # Phred quality score: https://en.wikipedia.org/wiki/Phred_quality_score
    def calc_base_quality(ascii_char):
        return 1-10**(-ord(ascii_char)/10)
    
    # Ista funkcija kao gore, samo sto se prosledi redni broj baze/read-a
    def calc_base_quality(self, base_num):
        # base_num < self.read_count !!!!
        return 1-10**(-ord(self.base_quality[base_num])/10)
    
    # Pomocna funkcija za racunanje broja pojavljivanja karaktera u stringu
    # Ovo sam mislila za prebrojavanje npr baza koje se poklapaju sa referentnom tj. broj . u read_results
    def num_of_occurence(self, char):
        return self.read_results.count(char)
    
    def count(self):
        #treba resiti pocetak i kraj stringa tj ^neki_znak i neki_znak$ ignorisati
        
        #sa ovim ispod izvrsavanje koda traje 28sekundi
        #posle ^ moze da ide bilo koji karakter, ukljucujuci i one bitne pa sa ovom linijom to brisem da ne bismo pogresno racunale
        #self.read_results = re.sub(r"\^.",'',self.read_results) 

        
        num_C = self.read_results.count('g') + self.read_results.count('C')
        num_G = self.read_results.count('c') + self.read_results.count('G')
        num_A = self.read_results.count('t') + self.read_results.count('A')
        num_T = self.read_results.count('a') + self.read_results.count('T')
        num_match = self.read_results.count('.') + self.read_results.count(',')
        num_deletions = self.read_results.count('*') + self.read_results.count('-')
        num_insertions = self.read_results.count('+')
        
        if self.ref_base == 'C':
            num_C += num_match
        elif self.ref_base == 'G':
            num_G += num_match
        elif self.ref_base == 'A':
            num_A += num_match
        elif self.ref_base == 'T':
            num_T += num_match
        


In [11]:
with open('/sbgenomics/project-files/merged-normal_pileup.pileup', 'r') as pileup_file:
    start = time.time()
    i = 0
    for line in pileup_file:
        a = Pileup_line(line)
        '''print(line)
        print(a.num_of_occurence(','))
        i+=1
        if i==30: 
            break'''
        a.count()
    end = time.time()
    print(end-start)

28.33063554763794


In [17]:
#https://pyvcf.readthedocs.io/en/v0.6.0/_modules/vcf.html
'''vcf_reader = vcf.Reader(open('/sbgenomics/project-files/mpileupvcf.mpileup.called.vcf', 'r'))
for record in vcf_reader:
    print(record.REF)'''
    
vcf_reader = vcf.Reader(filename='/sbgenomics/project-files/mpileupvcf.mpileup.called.vcf')
vcf_writer = vcf.Writer(open('/sbgenomics/project-files/proba_sloboda.txt', 'w'), vcf_reader)
for record in vcf_reader:
    vcf_writer.write_record(record)
    break

OSError: [Errno 30] Read-only file system: '/sbgenomics/project-files/proba_sloboda.txt'